In [1]:
import pandas as pd
import numpy as np
import datetime as datetime
import os
import sys
import time
from datetime import datetime
import datetime
import matplotlib.pyplot as plt
import math
import json


with open('user_input1.json', 'r') as f:
    data1 = json.load(f)
user_input1 = data1.get("user_input1", "No input found")
with open('user_input2.json', 'r') as f:
    data2 = json.load(f)
user_input2 = data2.get("user_input2", "No input found")
with open('user_input3.json', 'r') as f:
    data3 = json.load(f)
user_input3 = data3.get("user_input3", "No input found")
with open('user_input4.json', 'r') as f:
    data4 = json.load(f)
user_input4 = data4.get("user_input4", "No input found")


city_headway=str(user_input1)
city_peak_headway=str(user_input2)
Break_after_each_trip=user_input3
Lunch_break=str(user_input4)

df_314=pd.DataFrame(columns=["Route_number","Direction","New_start_time","Running_time","New_end_time","Break_time","dummy_id","Start_location","End_location","trip_number","Depot","Distance"])
df_315=pd.DataFrame(columns=["trip_id","arrival_time","departure_time","stop_id","stop_sequence"])
df_316=pd.DataFrame(columns=["route_id","service_id","trip_id","trip_headsign","shape_id"])
df_317=pd.DataFrame(columns=["Depot","Route_number","dummy_id","trip_number","Start_location","End_location","New_start_time","New_end_time","Distance"])


df=pd.read_excel("Route_inputs_sheet.xlsx")
df_elec=pd.read_excel("electrification_parameters.xlsx")


df_st=pd.read_csv("stops.csv")

for sk in range(len(df)):
    
#     df.head()


    first_trip_start_time=str(df.iloc[sk]["First trip time"])
    new_first_trip_time=str(df.iloc[sk]["First trip time"])


    from datetime import datetime
    i1=datetime.strptime(city_headway,'%H:%M:%S')
    city_headway_seconds = i1.second + i1.minute*60 + i1.hour*3600
#     city_headway_seconds

    UP_RT=datetime.strptime(str(df.iloc[sk]["UP_trip_time"]),'%H:%M:%S').second + datetime.strptime(str(df.iloc[sk]["UP_trip_time"]),'%H:%M:%S').minute*60 + datetime.strptime(str(df.iloc[sk]["UP_trip_time"]),'%H:%M:%S').hour*3600
#     UP_RT

    P_UP_RT=datetime.strptime(str(df.iloc[sk]["Peak_hour_UP_trip_time"]),'%H:%M:%S').second + datetime.strptime(str(df.iloc[sk]["Peak_hour_UP_trip_time"]),'%H:%M:%S').minute*60 + datetime.strptime(str(df.iloc[sk]["Peak_hour_UP_trip_time"]),'%H:%M:%S').hour*3600
#     P_UP_RT

    small_break_seconds=datetime.strptime(Break_after_each_trip,'%H:%M:%S').second + datetime.strptime(Break_after_each_trip,'%H:%M:%S').minute*60 + datetime.strptime(Break_after_each_trip,'%H:%M:%S').hour*3600
#     small_break_seconds

    lunch_break_seconds=datetime.strptime(Lunch_break,'%H:%M:%S').second + datetime.strptime(Lunch_break,'%H:%M:%S').minute*60 + datetime.strptime(Lunch_break,'%H:%M:%S').hour*3600
#     lunch_break_seconds


    # In[112]:


    # from datetime import datetime
    df_1=pd.DataFrame({"Route_number":[str(df.iloc[sk]["Route Number"])]*int((1440)/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"Direction":["UP"]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"Start_time":[first_trip_start_time]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"Running_time":[str(df.iloc[sk]["UP_trip_time"])]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"End_time":np.arange(int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute))})


    # In[113]:


    import datetime
    a=pd.to_datetime(df_1.iloc[0]["Start_time"]).date()
    time_00=datetime.datetime(year=a.year, month=a.month, day=a.day, hour=0, minute=0, second=0)
    y=pd.Timestamp(time_00)
    time_zero=y.timestamp()
#     time_zero


    # In[114]:


    new_start_time=[]
    import datetime
    for r in range(len(df_1)):
        if r==0:
            new_start_time.append(df_1.iloc[0]["Start_time"])
            continue
        else:
            l=(pd.to_datetime(df_1.iloc[0]["Start_time"]).timestamp()+city_headway_seconds*r)
            m=pd.to_datetime(datetime.datetime.utcfromtimestamp(l).strftime('%Y-%m-%d %H:%M:%S')).time()
            new_start_time.append(m)
    df_1["New_start_time"]=new_start_time
            


    # In[115]:


    import datetime
    s=pd.to_datetime(df_1['New_start_time'].astype("str"))
    s1=[]
    for k in range(len(s)):
        s1.append(s[k].timestamp())
    df_1["New_start_timestamp"]=s1


    # In[116]:


    new_end_timestamp=[]
    for k in range(len(df_1)):
        if (str(df_1.iloc[k]["New_start_time"])>="07:00:00" and str(df_1.iloc[k]["New_start_time"])<="11:00:00") or (str(df_1.iloc[k]["New_start_time"])>="16:00:00" and str(df_1.iloc[k]["New_start_time"])<="20:00:00"):
            new_end_timestamp.append(df_1.iloc[k]["New_start_timestamp"]+P_UP_RT)
            #print(new_end_timestamp)
        else:
             new_end_timestamp.append(df_1.iloc[k]["New_start_timestamp"]+UP_RT)
    df_1["New_end_timestamp"]=new_end_timestamp


    # In[117]:


    df_1["New_running_timestamp"]=df_1["New_end_timestamp"]-df_1["New_start_timestamp"]


    # In[118]:


    new_end_time=[]
    for i in range(len(df_1)):
        t=df_1["New_end_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_end_time.append(b.time())
    df_1['New_end_time']=new_end_time


    # In[119]:


    new_running_time=[]
    for i in range(len(df_1)):
        t=df_1["New_running_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_running_time.append(b.time())
    df_1['new running time']=new_running_time


    # In[120]:


#     df_1.head()


    # ## For different headway in peak hours

    # In[121]:


    from datetime import datetime
    i1=datetime.strptime(city_peak_headway,'%H:%M:%S')
    city_peak_headway_seconds = i1.second + i1.minute*60 + i1.hour*3600
#     city_peak_headway_seconds


    # In[122]:


    from datetime import datetime
    df_11=pd.DataFrame({"Route_number":[str(df.iloc[sk]["Route Number"])]*int((1440)/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"Direction":["UP"]*int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"Start_time":[first_trip_start_time]*int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"Running_time":[str(df.iloc[sk]["UP_trip_time"])]*int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"End_time":np.arange(int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute))})


    # In[123]:


    import datetime
    a=pd.to_datetime(df_11.iloc[0]["Start_time"]).date()
    time_00=datetime.datetime(year=a.year, month=a.month, day=a.day, hour=0, minute=0, second=0)
    y=pd.Timestamp(time_00)
    time_zero=y.timestamp()
#     time_zero


    # In[124]:


    new_start_time=[]
    import datetime
    for r in range(len(df_11)):
        if r==0:
            new_start_time.append(df_11.iloc[0]["Start_time"])
            continue
        else:
            l=(pd.to_datetime(df_11.iloc[0]["Start_time"]).timestamp()+city_peak_headway_seconds*r)
            m=pd.to_datetime(datetime.datetime.utcfromtimestamp(l).strftime('%Y-%m-%d %H:%M:%S')).time()
            new_start_time.append(m)
    df_11["New_start_time"]=new_start_time
            


    # In[125]:


    import datetime
    s=list(pd.to_datetime(df_11['New_start_time'].astype("str")))
    s11=[]
    for k in range(len(s)):
        s11.append(s[k].timestamp())
    df_11["New_start_timestamp"]=s11


    # In[126]:


    new_end_timestamp=[]
    for k in range(len(df_11)):
        if (str(df_11.iloc[k]["New_start_time"])>="07:00:00" and str(df_11.iloc[k]["New_start_time"])<="11:00:00") or (str(df_11.iloc[k]["New_start_time"])>="16:00:00" and str(df_11.iloc[k]["New_start_time"])<="20:00:00"):
            new_end_timestamp.append(df_11.iloc[k]["New_start_timestamp"]+P_UP_RT)
            #print(new_end_timestamp)
        else:
             new_end_timestamp.append(df_11.iloc[k]["New_start_timestamp"]+UP_RT)
    df_11["New_end_timestamp"]=new_end_timestamp


    # In[127]:


    df_11["New_running_timestamp"]=df_11["New_end_timestamp"]-df_11["New_start_timestamp"]


    # In[128]:


    new_end_time=[]
    for i in range(len(df_11)):
        t=df_11["New_end_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_end_time.append(b.time())
    df_11['New_end_time']=new_end_time


    # In[129]:


    new_running_time=[]
    for i in range(len(df_11)):
        t=df_11["New_running_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_running_time.append(b.time())
    df_11['new running time']=new_running_time


    # In[130]:


    df_11["New_start_time"]=df_11["New_start_time"].astype("str")

    df_12=df_11.loc[((df_11.New_start_time >='07:00:00' )& (df_11.New_start_time<= '11:00:00'))|
            ((df_11.New_start_time >='16:00:00' )& (df_11.New_start_time<= '20:00:00'))]

#     df_12.head()


    # In[131]:


    df_1["lab"]=[i+1 for i in range(len(df_1))]
#     df_1.head()


    # In[132]:


    df_1["New_start_time"]=df_1["New_start_time"].astype("str")
    df_13=df_1.loc[((df_1.New_start_time >='07:00:00' )& (df_1.New_start_time<= '11:00:00'))|
            ((df_1.New_start_time >='16:00:00' )& (df_1.New_start_time<= '20:00:00'))]


    # In[133]:


    el_lab=list(df_13["lab"].unique())
    #el_lab


    # In[134]:


    df_14=df_1[~df_1["lab"].isin(el_lab)]
    df_14 = df_14.drop('lab', axis=1)
#     df_14.head()


    # In[135]:


    df_15=pd.concat([df_12,df_14])
#     df_15.head()


    # ## Creating DOWN direction trips

    # In[136]:


    from datetime import datetime
    df_2=pd.DataFrame({"Route_number":[str(df.iloc[sk]["Route Number"])]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"Direction":["DOWN"]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"Start_time":[first_trip_start_time]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"Running_time":[str(df.iloc[sk]["UP_trip_time"])]*int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute)
                       ,"End_time":np.arange(int(1440/datetime.strptime(city_headway,'%H:%M:%S').minute))})


    # In[137]:


    from datetime import datetime
    DOWN_RT=datetime.strptime(str(df.iloc[sk]["DOWN_trip_time"]),'%H:%M:%S').second + datetime.strptime(str(df.iloc[sk]["DOWN_trip_time"]),'%H:%M:%S').minute*60 + datetime.strptime(str(df.iloc[sk]["DOWN_trip_time"]),'%H:%M:%S').hour*3600
#     DOWN_RT

    P_DOWN_RT=datetime.strptime(str(df.iloc[sk]["Peak_hour_DOWN_trip_time"]),'%H:%M:%S').second + datetime.strptime(str(df.iloc[sk]["Peak_hour_DOWN_trip_time"]),'%H:%M:%S').minute*60 + datetime.strptime(str(df.iloc[sk]["Peak_hour_DOWN_trip_time"]),'%H:%M:%S').hour*3600
#     P_DOWN_RT


    # In[138]:


    new_start_time=[]
    import datetime
    for r in range(len(df_2)):
        if r==0:
            new_start_time.append(df_2.iloc[0]["Start_time"])
            continue
        else:
            l=(pd.to_datetime(df_2.iloc[0]["Start_time"]).timestamp()+city_headway_seconds*r)
            m=pd.to_datetime(datetime.datetime.utcfromtimestamp(l).strftime('%Y-%m-%d %H:%M:%S')).time()
            new_start_time.append(m)
    df_2["New_start_time"]=new_start_time
            


    # In[139]:


    import datetime
    s=pd.to_datetime(df_2['New_start_time'].astype("str"))
    s1=[]
    for k in range(len(s)):
        s1.append(s[k].timestamp())
    df_2["New_start_timestamp"]=s1


    # In[140]:


    new_end_timestamp=[]
    for k in range(len(df_2)):
        if (str(df_2.iloc[k]["New_start_time"])>="07:00:00" and str(df_2.iloc[k]["New_start_time"])<="11:00:00") or (str(df_2.iloc[k]["New_start_time"])>="16:00:00" and str(df_2.iloc[k]["New_start_time"])<="20:00:00"):
            new_end_timestamp.append(df_2.iloc[k]["New_start_timestamp"]+P_DOWN_RT)
            #print(new_end_timestamp)
        else:
             new_end_timestamp.append(df_2.iloc[k]["New_start_timestamp"]+DOWN_RT)
    df_2["New_end_timestamp"]=new_end_timestamp


    # In[141]:


    df_2["New_running_timestamp"]=df_2["New_end_timestamp"]-df_2["New_start_timestamp"]


    # In[142]:


    new_end_time=[]
    for i in range(len(df_2)):
        t=df_2["New_end_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_end_time.append(b.time())
    df_2['New_end_time']=new_end_time


    # In[143]:


    new_running_time=[]
    for i in range(len(df_2)):
        t=df_2["New_running_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_running_time.append(b.time())
    df_2['new running time']=new_running_time


    # In[144]:


#     #city_peak_headway="00:03:00"
#     from datetime import datetime
#     i1=datetime.strptime(city_peak_headway,'%H:%M:%S')
#     city_peak_headway_seconds = i1.second + i1.minute*60 + i1.hour*3600
#     city_peak_headway_seconds


    # In[145]:


    from datetime import datetime
    df_21=pd.DataFrame({"Route_number":[str(df.iloc[sk]["Route Number"])]*int((1440)/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"Direction":["DOWN"]*int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"Start_time":[first_trip_start_time]*int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"Running_time":[str(df.iloc[sk]["UP_trip_time"])]*int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute)
                       ,"End_time":np.arange(int(1440/datetime.strptime(city_peak_headway,'%H:%M:%S').minute))})


    # In[146]:


    import datetime
    a=pd.to_datetime(df_21.iloc[0]["Start_time"]).date()
    time_00=datetime.datetime(year=a.year, month=a.month, day=a.day, hour=0, minute=0, second=0)
    y=pd.Timestamp(time_00)
    time_zero=y.timestamp()
#     time_zero


    # In[147]:


    new_start_time=[]
    import datetime
    for r in range(len(df_21)):
        if r==0:
            new_start_time.append(df_21.iloc[0]["Start_time"])
            continue
        else:
            l=(pd.to_datetime(df_21.iloc[0]["Start_time"]).timestamp()+city_peak_headway_seconds*r)
            m=pd.to_datetime(datetime.datetime.utcfromtimestamp(l).strftime('%Y-%m-%d %H:%M:%S')).time()
            new_start_time.append(m)
    df_21["New_start_time"]=new_start_time
            


    # In[148]:


    import datetime
    s=list(pd.to_datetime(df_21['New_start_time'].astype("str")))
    s11=[]
    for k in range(len(s)):
        s11.append(s[k].timestamp())
    df_21["New_start_timestamp"]=s11


    # In[149]:


    new_end_timestamp=[]
    for k in range(len(df_21)):
        if (str(df_21.iloc[k]["New_start_time"])>="07:00:00" and str(df_21.iloc[k]["New_start_time"])<="11:00:00") or (str(df_21.iloc[k]["New_start_time"])>="16:00:00" and str(df_21.iloc[k]["New_start_time"])<="20:00:00"):
            new_end_timestamp.append(df_21.iloc[k]["New_start_timestamp"]+P_UP_RT)
            #print(new_end_timestamp)
        else:
             new_end_timestamp.append(df_21.iloc[k]["New_start_timestamp"]+UP_RT)
    df_21["New_end_timestamp"]=new_end_timestamp


    # In[150]:


    df_21["New_running_timestamp"]=df_21["New_end_timestamp"]-df_21["New_start_timestamp"]


    # In[151]:


    new_end_time=[]
    for i in range(len(df_21)):
        t=df_21["New_end_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_end_time.append(b.time())
    df_21['New_end_time']=new_end_time


    # In[152]:


    new_running_time=[]
    for i in range(len(df_21)):
        t=df_21["New_running_timestamp"][i]
        #print(t)
        a=datetime.datetime.utcfromtimestamp(t).strftime('%Y-%m-%d %H:%M:%S')
        b=pd.to_datetime(a)
        new_running_time.append(b.time())
    df_21['new running time']=new_running_time


    # In[153]:


    df_21["New_start_time"]=df_21["New_start_time"].astype("str")

    df_22=df_21.loc[((df_21.New_start_time >='07:00:00' )& (df_21.New_start_time<= '11:00:00'))|
            ((df_21.New_start_time >='16:00:00' )& (df_21.New_start_time<= '20:00:00'))]

    # df_22.head()


    # In[154]:


    df_2["lab"]=[i+1 for i in range(len(df_2))]
    # df_2.head()


    # In[155]:


    df_2["New_start_time"]=df_2["New_start_time"].astype("str")


    # In[156]:


    df_23=df_1.loc[((df_2.New_start_time >='07:00:00' )& (df_2.New_start_time<= '11:00:00'))|
            ((df_2.New_start_time >='16:00:00' )& (df_2.New_start_time<= '20:00:00'))]


    # In[157]:


    el_lab=list(df_23["lab"].unique())
    #el_lab


    # In[158]:


    df_24=df_2[~df_2["lab"].isin(el_lab)]
    df_24 = df_24.drop('lab', axis=1)
    # df_24.head()


    # In[159]:


    df_25=pd.concat([df_22,df_24])
    # df_25


    # In[160]:


    #This will be final Form 3
    df_3=pd.concat([df_25,df_15]).sort_values(["New_start_timestamp"])


    # # Assignning  buses 

    # In[161]:


    No_of_total_buses=df.iloc[sk]['# buses running on route']
#     No_of_total_buses


    # In[162]:


    total_bus_numbers=[i+1 for i in range((No_of_total_buses))]


    # In[163]:


    Consistent_headway=int((DOWN_RT+UP_RT+2*small_break_seconds)/(No_of_total_buses))
#     Consistent_headway


    # In[164]:


    df_3["label"]=np.arange(1,len(df_3)+1)


    # In[165]:


    #made change here according to 2A
    cut=list(df_3[df_3["New_start_time"].astype("str")>=new_first_trip_time]["label"])[0]


    # In[166]:


#     cut


    # In[167]:


    sort_label=[]
    Sorted_start_timestamp=[]
    Sorted_end_timestamp=[]
    for i in range(len(df_3)):
        if df_3.iloc[i]["label"]<cut:
            sort_label.append(df_3.iloc[i]["label"]+len(df_3))
            Sorted_start_timestamp.append(df_3.iloc[i]["New_start_timestamp"]+24*3600)
            Sorted_end_timestamp.append(df_3.iloc[i]["New_end_timestamp"]+24*3600)
        else:
            sort_label.append(df_3.iloc[i]["label"])
            Sorted_start_timestamp.append(df_3.iloc[i]["New_start_timestamp"])
            Sorted_end_timestamp.append(df_3.iloc[i]["New_end_timestamp"])
    df_3["Sort_label"]=sort_label
    df_3["Sorted_start_timestamp"]=Sorted_start_timestamp
    df_3["Sorted_end_timestamp"]=Sorted_end_timestamp


    # In[168]:


    df_3=df_3.sort_values(["Sort_label"])


    # In[169]:


    df_3["New_start_time"]=df_3["New_start_time"].astype("str")
    df_3["New_end_time"]=df_3["New_end_time"].astype("str")


    # ### Assigning buses 

    # In[170]:


    df_31=df_3.loc[(df_3.New_start_time>=new_first_trip_time)& (df_3.New_end_time<"23:55:00")]


    # In[171]:


#     df_31


    # In[172]:


    No_of_total_buses=list(np.arange(1,No_of_total_buses+1))


    # In[173]:


    Bus_upper_limit_sec=[(16)*3600 for i in No_of_total_buses]


    # In[174]:


    num_bus_lower=0
    num_bus_upper=num_bus_lower+len(No_of_total_buses)
#     num_bus_upper


    # In[175]:


    lunch_break_after=4*3600


    # In[176]:


    done_trips=[]
    bus_and_trips=dict()
    done_count=0
    label_id={}
    for b in No_of_total_buses:
        df4=df_31[df_31["label"].isin(done_trips)==False]
        df4=df4.sort_values(["Sorted_start_timestamp","Direction"],ascending=[True, True])
        bus_and_trips[b]=[] 
        #print(len(bus_and_trips))
        if b==1:
            ltd=[df4.iloc[0]['label'],df4.iloc[0]['Sorted_start_timestamp'],df4.iloc[0]['Sorted_end_timestamp'],df4.iloc[0]["Direction"]]
            first_bus_start_timestamp=df4.iloc[0]["Sorted_start_timestamp"]
            fisrt_bus_direction=ltd[3]
            print(fisrt_bus_direction)

        else:
    #         ltd=[df4.iloc[1]['label'],df4.iloc[1]['Sorted_start_timestamp'],df4.iloc[1]['Sorted_end_timestamp'],df4.iloc[1]["Direction"]]
    #         first_bus_start_timestamp=df4.iloc[0]["Sorted_start_timestamp"]
    #         fisrt_bus_direction=ltd[3]
    #         print(fisrt_bus_direction)
            l=first_bus_start_timestamp+(b-1)*Consistent_headway
            for k in range(len(df4)):
                if df4.iloc[k]["Sorted_start_timestamp"]>=l:
                    ltd=[df4.iloc[k]['label'],df4.iloc[k]['Sorted_start_timestamp'],df4.iloc[k]['Sorted_end_timestamp'],df4.iloc[k]["Direction"]]
                    break
        start_timestamp=ltd[1]
        max_limit=start_timestamp+Bus_upper_limit_sec[b-1]
        CRT=0
        Long_break_cum_time=0

        for i in range(len(df4)):
            if df4.iloc[i]["Sorted_start_timestamp"] >= ltd[1] and len(bus_and_trips[b])==0 and df4.iloc[i]["Direction"]==fisrt_bus_direction :#and df4.iloc[i]["Sorted_end_timestamp"]<=max_limit:
                #print(df4.iloc[i]["label"])
                #print("Check1")
                bus_and_trips[b].append(ltd)
                done_trips.append(df4.iloc[i]["label"])
                ltd=[df4.iloc[i]['label'],df4.iloc[i]['Sorted_start_timestamp'],df4.iloc[i]['Sorted_end_timestamp'],df4.iloc[i]["Direction"]]
                done_count+=1
                label_id[df4.iloc[i]["label"]]=b
                CRT=df4.iloc[i]["Sorted_end_timestamp"]-start_timestamp
                Long_break_cum_time=Long_break_cum_time+df4.iloc[i]["New_running_timestamp"]+small_break_seconds
                #print(CRT)
            elif df4.iloc[i]["Sorted_start_timestamp"]>=ltd[2]+small_break_seconds and CRT<=Bus_upper_limit_sec[b-1] and Long_break_cum_time<lunch_break_after and df4.iloc[i]["Direction"]!=ltd[3] and df4.iloc[i]["Sorted_end_timestamp"]<=max_limit :
                #print("Check2")
                bus_and_trips[b].append(ltd)
                done_trips.append(df4.iloc[i]["label"])
                ltd=[df4.iloc[i]['label'],df4.iloc[i]['Sorted_start_timestamp'],df4.iloc[i]['Sorted_end_timestamp'],df4.iloc[i]["Direction"]]
                done_count+=1
                label_id[df4.iloc[i]["label"]]=b
                CRT=df4.iloc[i]["Sorted_end_timestamp"]-start_timestamp
                Long_break_cum_time=Long_break_cum_time+df4.iloc[i]["New_running_timestamp"]+small_break_seconds
                #print(b)
                #print(Long_break_time)
                #print(CRT)
            elif df4.iloc[i]["Sorted_start_timestamp"]>=ltd[2]+lunch_break_seconds and CRT<=Bus_upper_limit_sec[b-1] and Long_break_cum_time>=lunch_break_after and df4.iloc[i]["Direction"]!=ltd[3] and df4.iloc[i]["Sorted_end_timestamp"]<=max_limit:
                #print("Check3")
                Long_break_cum_time=0
                bus_and_trips[b].append(ltd)
                done_trips.append(df4.iloc[i]["label"])
                ltd=[df4.iloc[i]['label'],df4.iloc[i]['Sorted_start_timestamp'],df4.iloc[i]['Sorted_end_timestamp'],df4.iloc[i]["Direction"]]
                done_count+=1
                label_id[df4.iloc[i]["label"]]=b
                CRT=df4.iloc[i]["Sorted_end_timestamp"]-start_timestamp
                #Long_break_cum_time=0
                Long_break_cum_time=Long_break_cum_time+df4.iloc[i]["New_running_timestamp"]+small_break_seconds
                #print(CRT)


    # In[177]:


    f1=[]
    for i in range(len(df_31)):
        try:
            k=label_id[df_31.iloc[i]["label"]]
            f1.append(k)
        except:
            f1.append("Can not be completed with given number of buses and constraints")
    df_31["dummy_id"]=f1


    # In[178]:


    df_31=df_31.sort_values(["dummy_id","Sorted_start_timestamp"])


    # In[273]:


    #df_30=df_3[df_3["dummy_id"]!="Can not be completed with given number of buses and constraints"]
    
#     df_31["Shift Type"]="Day Shift"
    df_31["Depot"]="Depot "+str(sk+1)


    # In[274]:


#     df_31.head()


    # In[181]:


#     df_31["dummy_id"].unique()


    # In[ ]:





    # ### Checking for last return trips 

    # In[182]:


    remove_labels=[]
    for i in range(len(df_31)-1):
        if df_31.iloc[i]["dummy_id"]!=df_31.iloc[i+1]["dummy_id"]:
            if df_31.iloc[i]["Direction"]!="UP":
    #             print(df_31.iloc[i]["dummy_id"])
    #             print(df_31.iloc[i]["label"])
    #             print(i)
                df_31.iat[i,15] ="Can not be completed with given number of buses and constraints"
                remove_labels.append(df_31.iloc[i]["label"])
    done_trips=list(set(done_trips)-set(remove_labels))


    # In[183]:


    df_31=df_31[df_31["dummy_id"]!="Can not be completed with given number of buses and constraints"]


    # In[184]:


    trip_number=[]
    done_buses=[]
    for i in range(len(df_31)):
        if df_31.iloc[i]["dummy_id"] not in done_buses:
            k=1
            trip_number.append(k)
            done_buses.append(df_31.iloc[i]["dummy_id"])
        elif df_31.iloc[i]["dummy_id"] in done_buses:
            k=k+1
            trip_number.append(k)
    df_31["trip_number"]=trip_number


    # In[185]:


    start=[]
    end=[]
    for i in range(len(df_31)):
        if df_31.iloc[i]["Direction"]=="DOWN":
            start.append(df.iloc[sk]["Down_start_point_name"])
            end.append(df.iloc[sk]["Down_end_point_name"])
        else:
            start.append(df.iloc[sk]["Down_end_point_name"])
            end.append(df.iloc[sk]["Down_start_point_name"])
    df_31["Start_location"]=start
    df_31["End_location"]=end


    # In[186]:


#     df_31.head()


    # In[187]:


#     df_31.to_excel("p1_Check.xlsx",index=False)


    # In[ ]:





    # In[188]:


    bt=[]
    done_ids=[]
    for i in range(len(df_31)):
        if i==0:
            bt.append(0)
            done_ids.append(df_31.iloc[i]["dummy_id"])
            continue
        elif i==len(df_31)-1:
            bt.append(0)
        elif df_31.iloc[i]["dummy_id"] in done_ids:
            k=df_31.iloc[i]["Sorted_start_timestamp"]-df_31.iloc[i-1]["Sorted_end_timestamp"]
            bt.append(k)
    #         print("success")
        elif df_31.iloc[i]["dummy_id"] not in done_ids:
            done_ids.append(df_31.iloc[i]["dummy_id"])
            bt.append(0)
            


    # In[ ]:





    # In[189]:


#     len(df_31)


    # In[190]:


    df_31["break_time"]=bt


    # In[191]:


#     df_31.head()


    # In[192]:


    df_31["Break_time"]=pd.to_datetime(df_31["break_time"],unit="s").dt.strftime("%H:%M:%S")
    df_31["Running_time"]=pd.to_datetime(df_31["New_running_timestamp"],unit="s").dt.strftime("%H:%M:%S")


    # In[193]:


#     df_31.head()


    # In[194]:


    df_31["New_start_time"]=pd.to_datetime(df_31["New_start_timestamp"],unit="s").dt.strftime("%H:%M:%S")
    df_31["New_end_time"]=pd.to_datetime(df_31["New_end_timestamp"],unit="s").dt.strftime("%H:%M:%S")
    df_31["Distance"]=df.iloc[sk]["Distance"]

    # In[195]:
    df_32_1=df_31[["Depot","Route_number","dummy_id","trip_number","Start_location","End_location","New_start_time","New_end_time","Distance"]]

#     df_31.head()


    # In[196]:


    df_31[["Route_number","Direction","New_start_time","Running_time","New_end_time","Break_time","dummy_id","Start_location","End_location","Distance"]]


    # In[ ]:





    # In[197]:


    df_313=df_31[["Route_number","Direction","New_start_time","Running_time","New_end_time","Break_time","dummy_id","Start_location","End_location","trip_number","Depot","Distance"]]
    df_314=pd.concat([df_314,df_313])




    df_314.to_excel("Route_wise_schedule.xlsx",index=False)
    
    df_317=pd.concat([df_317,df_32_1])
    
    stops=dict(zip(df_st["stop_name"],df_st["id"]))
#     done_ids=[]
#     k=0
#     for i in range(len(df_31)):
#         if df_31.iloc[i]["dummy_id"] not in done_ids:
#             k=0
#             done_ids.append(df_31.iloc[i]["dummy_id"])
#             trip_id.append("WD"+str(df_31.iloc[i]["dummy_id"]))
#             arr_time.append(df_31.iloc[i]["New_start_timestamp"])
#             dep_time.append(df_31.iloc[i]["New_start_timestamp"])
#             stop_id.append(stops[df_31.iloc[i]["Start_location"]])
#             stop_seq.append(k+1)
#             k=k+1
#             continue
#         else:
#             trip_id.append("WD"+str(df_31.iloc[i]["dummy_id"]))
#             arr_time.append(df_31.iloc[i-1]["Sorted_end_timestamp"])
#             dep_time.append(df_31.iloc[i]["Sorted_start_timestamp"])
#             stop_id.append(stops[df_31.iloc[i]["Start_location"]])
#             stop_seq.append(k+1)
#             k=k+1
            
    df41_d=df_31[df_31["Direction"]=="DOWN"]
    down_start_pivot=pd.pivot_table(df41_d, values='New_start_timestamp', index=['Start_location'],
                       columns=['New_start_time'], aggfunc="sum")
    
    first_row_list=down_start_pivot.iloc[0].tolist()
    rows_df=[]
    for i in range(len(stops)-1):
        new_row=[x+(i*70) for x in first_row_list]
        rows_df.append(new_row)
    df_123=pd.DataFrame(rows_df,columns=list(down_start_pivot.columns))
    down_end_pivot=pd.pivot_table(df41_d, values='New_end_timestamp', index=['End_location'],
                       columns=['New_start_time'], aggfunc="sum")
    
    
    df_123.loc[len(df_123)]=down_end_pivot.iloc[0].tolist()
    df_123["stops"]=df_st["stop_name"]
    df_123.set_index('stops', inplace=True)
    
    for col in df_123.columns:
        df_123["Time_"+col]=pd.to_datetime(df_123[col],unit='s').dt.strftime("%H:%M:%S")
    
    df_123=df_123[[col for col in df_123.columns if col.startswith("Time_")]]
    trip_id=[]
    arr_time=[]
    dep_time=[]
    stop_id=[]
    stop_seq=[]
    
    t=0
    for c in df_123.columns:
        #print(c)
        t=t+1
        for i in range(len(df_123)):
            if i==0:
                trip_id.append("WD"+df.iloc[sk]["Route Number"]+str(t))
                arr_time.append(df_123.iloc[i][c])
                #print("Okay")
                dep_time.append(df_123.iloc[i][c])
                stop_seq.append(i+1)
                stop_id.append(stops[df_123.index[i]])
            else:
                trip_id.append("WD"+df.iloc[sk]["Route Number"]+str(t))
                arr_time.append(df_123.iloc[i-1][c])
                dep_time.append(df_123.iloc[i][c])
                stop_seq.append(i+1)
                stop_id.append(stops[df_123.index[i]])
                
    df_stop_times_down=pd.DataFrame([trip_id,arr_time,dep_time,stop_id,stop_seq], index=['trip_id', 'arrival_time',"departure_time","stop_id","stop_sequence"]).T

    df41_u=df_31[df_31["Direction"]=="UP"]
    
    up_start_pivot=pd.pivot_table(df41_u, values='New_start_timestamp', index=['Start_location'],
                       columns=['New_start_time'], aggfunc="sum")
    
    first_row_list=up_start_pivot.iloc[0].tolist()
    
    rows_df=[]
    for i in range(len(stops)-1):
        new_row=[x+(i*70) for x in first_row_list]
        rows_df.append(new_row)
    df_124=pd.DataFrame(rows_df,columns=list(up_start_pivot.columns))
    
    up_end_pivot=pd.pivot_table(df41_u, values='New_end_timestamp', index=['End_location'],
                       columns=['New_start_time'], aggfunc="sum")
    
    
    df_124.loc[len(df_124)]=up_end_pivot.iloc[0].tolist()
    
    df_st_rev=df_st.sort_values(by="seq",ascending=False).reset_index(drop=True)
    df_st_rev2=df_st_rev
    
    df_st_rev2["seq2"]=[x+1 for x in range(len(df_st_rev2)) ]
    df_124["stops"]=df_st_rev2["stop_name"]
    
    df_124.set_index('stops', inplace=True)
    
    for col in df_124.columns:
        df_124["Time_"+col]=pd.to_datetime(df_124[col],unit='s').dt.strftime("%H:%M:%S")
        
    df_124=df_124[[col for col in df_124.columns if col.startswith("Time_")]]
    
    trip_id=[]
    arr_time=[]
    dep_time=[]
    stop_id=[]
    stop_seq=[]
    t=0
    for c in df_124.columns:
        #print(c)
        t=t+500
#         print(t)
        for i in range(len(df_124)):
            if i==0:
                trip_id.append("WD"+df.iloc[sk]["Route Number"]+str(t))
                arr_time.append(df_124.iloc[i][c])
                #print("Okay")
                dep_time.append(df_124.iloc[i][c])
                stop_seq.append(i+1)
                stop_id.append(stops[df_124.index[i]])
            else:
                trip_id.append("WD"+df.iloc[sk]["Route Number"]+str(t))
                arr_time.append(df_124.iloc[i-1][c])
                dep_time.append(df_124.iloc[i][c])
                stop_seq.append(i+1)
                stop_id.append(stops[df_124.index[i]])
                
    df_stop_times=pd.DataFrame([trip_id,arr_time,dep_time,stop_id,stop_seq], index=['trip_id', 'arrival_time',"departure_time","stop_id","stop_sequence"]).T           
    
    df_stop_times=pd.concat([df_stop_times_down,df_stop_times])
    df_315=pd.concat([df_315,df_stop_times])
    
    df_315.to_csv("stop_times.txt",index=False)

    df42=df_stop_times["trip_id"].unique()
    df42=pd.DataFrame(list(df_stop_times["trip_id"].unique()),columns=["trip_id"])
    df42["route_id"]=df.iloc[sk]["Route Number"]
    df42["service_id"]="WD"
    df42["trip_headsign"]="Rjkt"
    df42["shape_id"]="sh37"
    df42=df42[["route_id","service_id","trip_id","trip_headsign","shape_id"]]
    
    df_316=pd.concat([df_316,df42])
    df_316.to_csv("trips.txt",index=False)
    
    
df_31=df_317
df_31["dummy_id"]=df_31["Route_number"]+"_"+df_31["dummy_id"].astype("str")

Dep=[]
Arrival=[]
for i in range(len(df_31)):
    k=str(df_31.iloc[i]["New_start_time"])
    hours,minutes,sec=map(int,k.split(':'))
    d=hours+minutes/60
    Dep.append(d)
    k=str(df_31.iloc[i]["New_end_time"])
    hours,minutes,sec=map(int,k.split(':'))
    d=hours+minutes/60
    Arrival.append(d)
df_31["Departure"]=Dep
df_31["Arrival"]=Arrival

df_31=df_31[["Depot","dummy_id","trip_number","Start_location","End_location","Departure","Arrival","Distance"]]

df_31.rename(columns={"dummy_id":"Route","trip_number":"Trip number","Start_location":"From","End_location":"To"},inplace=True)

df=df_31

gadbad = []
for r in list(df['Route'].unique()):
    temp = df[df['Route'] == r].sort_values(by = 'Trip number')
    for i in range(len(temp)-1):
        if temp.iloc[i]['Arrival'] > temp.iloc[i+1]['Departure']:
            gadbad.append(r)
            break
print(len(gadbad))
df = df[~df['Route'].isin(gadbad)]


df.sort_values(by = ['Route',"Trip number"]).reset_index(drop = True)

#Battery capacity KWh
B = df_elec.iloc[0][0]

#Reserve battery ratio = 20%
RBR = df_elec.iloc[0][1]/100

#Energy efficiency = 1.1 KWh per Km
EE = df_elec.iloc[0][2]

#Range = B(1 - RBR)/EE
R = (B*(1 - RBR))/EE  #Range in Kilometers


#KWh per Km
U = B/R

#Charging rate (KWh per Minute)
C =df_elec.iloc[0][3]

#Kilometer gain per minute of charging: (Km per Minute)
KPM = C/U


#Time required to charge the bus fully in hours
TOC = R/(KPM*60)

#Allowed waiting time including charging time in hours
W = df_elec.iloc[0][4]

#Cost per charger gun
CG = df_elec.iloc[0][5]

#Charging infrastructure fixed cost
FC = df_elec.iloc[0][6]

#Maximum charger allowed at a location
NC = df_elec.iloc[0][7]

#Charging rate of overnight chargers (KWh per Minute)
KPM_slow = df_elec.iloc[0][8]
U_slow = KPM_slow*B/R

#Allowable change in the schedule at any node
delta = df_elec.iloc[0][9]

#Charger capacity = 260 KW and At a stop Not more than 1 MW
#For depot --> 5.5 MW is allowed


Infeasible_Routes = list(df[df['Distance'] > R]['Route'].unique())
df = df[~df['Route'].isin(Infeasible_Routes)]
    
    
charger_status = {}        #Busy status of chargers during the day
Chargers = {}              #No. of chargers required at each stop

for i in list(set(df['From'].values)):
    if i not in charger_status:
        charger_status[i] = []
        Chargers[i] = 0
for i in list(set(df['To'].values)):
    if i not in charger_status:
        charger_status[i] = []
        Chargers[i] = 0
# Chargers

routes = list(df['Route'].unique())

depots = []
for depot in list(df['Depot'].unique()):
    depots.append(depot)
    Chargers[depot] = int(df_elec.iloc[0][10])
    charger_status[depot] = []
    for i in range(Chargers[depot]):
        charger_status[depot].append([True]*288)
        
        
routes_list = []
for i in routes:
    routes_list.append([float(df[df['Route'] == i]['Departure'].min()), i]) 
routes_list.sort()
    
Route_charging = pd.DataFrame(columns = ['Bus', 'Charged at', 'Charger_gun', 'Charge Start Time', 'Charging duration(Minutes)', 'Waiting'])  
    
Schedule_changes = pd.DataFrame(columns = ['Bus', 'Node', 'Arrival_time', 'Change in schedule(Mins)'])   
Charger_busy_status = {}    
Infeasible_Routes = []   
    
def to_time_units(time):
        hours = int(time)
        mins = math.ceil((time - hours)*60)
        return f"{hours}:{mins:02}"
        
def Priority(node):
        return max(len(df[df['From'] == node]), len(df[df['To'] == node]))
    
    
def find_charger(data):
        nodes = list(set(list(data['From'].unique()) + list(data['To'].unique())))
        for i in nodes:
            if(Chargers[i] != 0):
                return True
        return False
    
    
def to_decimal(time):
        lst = time.split(':')
        hour = int(lst[0])
        mins = math.floor(int(lst[1])/5)*5
        return hour + (mins/60)
    
def calc_avl_time2(data, tot_time_avl):
        node = data['Node']
        if Chargers[node] == 0:
            return 0
        start = math.ceil(data['Start']*12)
        end = math.ceil(data['End']*12)
        Range = data['Range_at_start']
        if node in depots:
            charging_rate = KPM_slow
            time_span = [True]*(288 - abs(end - start))
        else:
            charging_rate = KPM
            time_span = [True]*(end - start)
        
        #Calcualting available time for charging
        time_available = 0
        for charger_no in range(len(charger_status[node])):
            index = 0
            while index < len(time_span) and Range + ((tot_time_avl + 0)*charging_rate) < R:
                if start + index < 288:
                    time_stamp = start + index
                else:
                    time_stamp = start + index - 288
                if charger_status[node][charger_no][time_stamp] == True and time_span[index] == True:
                    time_available += 5
                    Range = min(Range + (5*charging_rate), R)
                    time_span[index] = False
                index += 1
        return time_available
    
def charge_existing(data_frame, min_charge, case):
        query = 'VIJAYAPURA-III Depot_30'
        flag = "None"
        max_time_availbale = -1000
        charge_index = -1
        potential_locs = []
        tot_time_avl = 0
        for i in range(data_frame.shape[0]):
            trip = data_frame.iloc[i]
            node = trip['Node']
            if node in depots:
                charging_rate = KPM_slow
            else:
                charging_rate = KPM
            
            if Chargers[node] != 0:
                ta1 = calc_avl_time2(trip, 0)
                
                if trip['charged_before'] == False and trip['Range_at_start'] + (ta1*charging_rate) >= trip['rem_dist']:
                    if trip['Route'] == query:
                        print("Time available for charging at only", node, ta1)
                    max_time_availbale = ta1
                    charge_index = i
                    flag = 'Full_trip'
                    break
                    
                else:
                    ta2 = 0
                    if trip['charged_before'] == False:
                        ta2 = calc_avl_time2(trip, tot_time_avl)
                        if trip['Route'] == query:
                            print('Time available for charging at more than one nodes.', [node, ta2])
                    potential_locs.append([i, tot_time_avl, ta2, 0])
                    tot_time_avl += ta2
                    flag = 'Partial_trip'
        
        if case == "New":
            additional_time = 0
            ta4 = 0
            for loc in potential_locs:
                ta4 += loc[2]
                if data_frame.iloc[loc[0]]['Schedule_changed'] == False:
    #                 change = min(math.ceil(((R - data_frame.iloc[loc[0]]['Range_at_start'])/charging_rate)/5)*5 - ta4, delta)
                    extra_charge = data_frame.iloc[loc[0]].copy()
                    extra_charge['Start'] = data_frame.iloc[loc[0]]['End']
                    extra_charge['End'] = extra_charge['Start'] + (delta/60)
    #                 display(extra_charge)
                    print(loc)
                    change = min(calc_avl_time2(extra_charge, loc[1] + loc[2]), delta)
                    additional_time += change
                    loc[3] = change
                if trip['Route'] == query:
                    print('Additional Time:', data_frame.iloc[loc[0]]['Node'], loc[2], loc[3])
        
        if flag == 'Full_trip':
            Charge(data_frame.iloc[charge_index], 0, 1e9)
            return (-1, flag, max_time_availbale)
        elif flag == 'Partial_trip' and tot_time_avl >= min_charge:
            temp_list = []
            ta3 = 0
            for loc in potential_locs:
                ta3 += loc[2]
                if data_frame.iloc[loc[0]]['Node'] in depots:
                    charging_rate = KPM_slow
                else:
                    charging_rate = KPM
                if loc[2] > 0:
                    temp_list.append((loc[0], loc[2]))
                    Charge(data_frame.iloc[loc[0]], loc[1], 1e9)
                if data_frame.iloc[loc[0]]['Range_at_start'] + (charging_rate*ta3) >= data_frame.iloc[loc[0]]['rem_dist']:
                    return (-1 , 'Full_trip', ta3)
                
            return (temp_list, flag, tot_time_avl)
        elif case == 'New' and flag == 'Partial_trip' and tot_time_avl + additional_time >= min_charge:
            temp_list = []
            ta5 = 0
            scedule_to_be_changed = math.ceil((min_charge - tot_time_avl)/5)*5
            if trip['Route'] == query:
                print('Total time available', tot_time_avl, 'schedule to be changed:', scedule_to_be_changed)
            for loc in potential_locs:
                if trip['Route'] == query:
                    print(data_frame.iloc[loc[0]]['Node'], loc[2], loc[3])
                if data_frame.iloc[loc[0]]['Schedule_changed'] == False:
                    if data_frame.iloc[loc[0]]['Node'] in depots:
                        charging_rate = KPM_slow
                    else:
                        charging_rate = KPM
                    
                    extra_charge = data_frame.iloc[loc[0]].copy()
                    extra_charge['Start'] = data_frame.iloc[loc[0]]['End']
                    extra_charge['End'] = extra_charge['Start'] + (delta/60)
                    extend = min(loc[3], scedule_to_be_changed, calc_avl_time2(extra_charge, loc[1] + loc[2]))
                    print("Time extended:", extend)
                    scedule_to_be_changed -= extend
                    ta5 += loc[2] + extend

                    if trip['Route'] == query:
                        print('Total charge till node:', data_frame.iloc[loc[0]]['Node'], ta5, '***')
                    temp_list.append((loc[0], loc[2], extend))
                    data_frame['Start'].iloc[loc[0]+1:] += extend/60
                    data_frame['End'].iloc[loc[0]+1:] += extend/60
                    if loc[2] > 0:
                        display(data_frame.iloc[loc[0]])
                        Charge(data_frame.iloc[loc[0]], loc[1], 1e9)
                    if extend > 0:
                        Charge(extra_charge, loc[1]+loc[2], extend)
    #                     print("***Le dekh le")
    #                     display(data_frame.iloc[loc[0]])
                        Schedule_changes.loc[len(Schedule_changes)] = [data_frame.iloc[loc[0]]['Route'], data_frame.iloc[loc[0]]['Node'], to_time_units(data_frame.iloc[loc[0]]['Start']), extend]
                    if data_frame.iloc[loc[0]]['Range_at_start'] + (charging_rate*ta5) >= data_frame.iloc[loc[0]]['rem_dist']:
                        return (temp_list , 'Full_trip', ta5)
            return (temp_list, 'Partial_with_changes', ta5)
        else:
            return (potential_locs,'NA', tot_time_avl)

        
def Charge(row, already_charged_time, max_time):
        query = 'VIJAYAPURA-III Depot_30'
        node = row['Node']
        start = math.ceil(row['Start']*12)
        end = math.floor(row['End']*12)
        if row['Route'] == query:
            print(row['Start'], row['End'], start, end)
        Range = row['Range_at_start'] + (already_charged_time*KPM)
        total_charge_time = 0
        
        if node in depots:
            charging_rate = KPM_slow
            time_span = [True]*(288 - abs(end - start))
        else:
            charging_rate = KPM
            time_span = [True]*(end-start)
        charge_time = 0
        for charger_no in range(len(charger_status[node])):
            charge_start = 100000
            index = 0
            while index < len(time_span) and Range + (0*charging_rate) < R and Range < row['rem_dist'] and total_charge_time < max_time:
                if start + index < 288:
                    time_stamp = start + index
                else:
                    time_stamp = start + index - 288
                if charger_status[node][charger_no][time_stamp] == True and time_span[index] == True:
                    charge_start = time_stamp if charge_start == 100000 else charge_start
                    charger_status[node][charger_no][time_stamp] = False
                    charge_time += 5
                    total_charge_time += 5
                    if row['Route'] == query:
                        print(time_stamp)
                    Range = min(Range + (5*charging_rate), R)
                    time_span[index] = False
                elif charge_time != 0:
                    Route_charging.loc[len(Route_charging)] = [row['Route'], node, charger_no + 1, \
                    to_time_units(charge_start/12), charge_time, math.ceil((((charge_start/12) - row['Start'])*60)/5)*5]
                    charge_time = 0
                    charge_start = 100000
                index += 1
            if charge_time != 0:
                Route_charging.loc[len(Route_charging)] = [row['Route'], node, charger_no + 1, \
                to_time_units(charge_start/12), charge_time, math.ceil((((charge_start/12) - row['Start'])*60)/5)*5]
                charge_time = 0
                charge_start = 100000
                

def reverse_the_charge(bus):
    query = 'VIJAYAPURA-III Depot_79'
    depot_charge_row = Route_charging[(Route_charging['Bus'] == bus)]
    depot_charge_row['Charge Start Time'] = depot_charge_row['Charge Start Time'].apply(to_decimal)
        
    for index in range(len(depot_charge_row)):
            
        initial = math.ceil(depot_charge_row.iloc[index]['Charge Start Time']*12)
        final = math.ceil((depot_charge_row.iloc[index]['Charge Start Time'] \
                       + (depot_charge_row.iloc[index]['Charging duration(Minutes)']/60))*12)
        if bus == query:
            print("Charge reversed",initial, final)
        for tstamp in range(initial, final+1):
            if tstamp >= 288:
                tstamp = tstamp - 288
            charger_status[depot_charge_row.iloc[index]['Charged at']][depot_charge_row.iloc[index]['Charger_gun'] - 1][tstamp] = True
    Route_charging.drop(list(depot_charge_row.index), inplace = True)
    Route_charging.reset_index(drop = True, inplace = True)

    
    
# query = 'VIJAYAPURA-III Depot_30'
query = 'Depot 3'
for element in routes_list:
        j = element[1]
        r = df[df['Route'] == j]
        r = r.sort_values(by='Departure')
        Range_at_start = 0
        if(r['Distance'].sum() < Range_at_start):
            continue
        rem_dist = r['Distance'].sum()
        
        vis_nodes = pd.DataFrame(columns = ['Route', 'Node', 'Priority', 'Range_at_start', 'rem_dist', 'Start', 'End', 'time_available', 'charged_before', 'Schedule_changed'])
        vis_nodes.loc[0] = [j, r.iloc[0]['Depot'], 0, Range_at_start, r['Distance'].sum(), r.iloc[len(r) - 1]['Arrival'], r.iloc[0]['Departure'], \
                            min(math.ceil(((R - Range_at_start)/KPM_slow)/5)*5, math.ceil(((r.iloc[0]['Departure'] - r.iloc[len(r) - 1]['Arrival'])*60)/5)*5), False, False]
        
        i = 0
        Schedule_changed_indexes = []
        while i < r.shape[0]:
            if i == 0:
                Schedule_changed = False
            if(i > 0):
                prev_trip = r.iloc[i-1]
            trip = r.iloc[i]
            if i != r.shape[0] - 1: 
                next_trip = r.iloc[i+1]
            start_node = trip["From"]
            end_node = trip["To"]
            print(start_node, Range_at_start)
            
            if Range_at_start < trip['Distance']:
                if i == 0:
                    charging_rate = KPM_slow
                else:
                    charging_rate = KPM
                time_req =  (trip['Distance'] - Range_at_start)/charging_rate
                existing = charge_existing(vis_nodes, time_req, 'Existing')
                if existing[1] == 'Full_trip':
                    if j == query:
                        print("**AA GAYA - 1**")
                    break
                elif existing[1] == 'Partial_trip':
                    if j == query:
                        print("**AA GAYA - 2**")
                    for temp in existing[0]:
                        vis_nodes['charged_before'].iloc[temp[0]] = True
                        vis_nodes['Range_at_start'].iloc[temp[0]:] += (charging_rate*temp[1])
                        vis_nodes["Range_at_start"] = np.where(vis_nodes["Range_at_start"] > R, R, vis_nodes["Range_at_start"])
                    Range_at_start = min(Range_at_start + (charging_rate*existing[2]), R) - trip['Distance']
                else:
                    if j == query:
                        print("***Condition-3***", 'Time required:', time_req)
                    temp_df = vis_nodes.sort_values(by = ['Priority', 'time_available'], ascending = [False, False])
                    temp_df = temp_df[temp_df['charged_before'] == False]
    #                 temp_df.drop_duplicates(subset="Priority", keep= 'first', inplace=True)
                    temp_df.reset_index(inplace = True)
                    new_charger_index = -1
                    vis_nodes_ind = -1
                    
                    print(j, len(temp_df), "***ThisOne")
                    for threshold in range(1, len(temp_df)+1):
                        print('threshold:', threshold)
                        new_chargers = 0
                        new_chargers_list = []
                        for index in range(len(temp_df)):
                            row = temp_df.iloc[index]
                            time_avl = row['time_available']
                            if j == query:
                                    print('Chargers available at node:', row['Node'], Chargers[row['Node']])
                            already_avl_time = calc_avl_time2(vis_nodes.iloc[int(row['index'])], 0)
                            if Chargers[row['Node']] < NC and time_avl >= time_req:
                                if j == query:
                                    print('Allocating new charger at', row['Node'])
                                new_charger_index = index
                                vis_nodes_ind = int(row['index'])
                                break

                            elif Chargers[row['Node']] < NC:
                                if j == query:
                                    print('Condition-1', row['Node'], Chargers[row['Node']])

                                if time_avl > already_avl_time:
                                    if j == query:
                                        print('Trying by giving a new charger at', row['Node'])
                                    Chargers[row['Node']] += 1
                                    charger_status[row['Node']].append([True]*288)
                                    new_chargers += 1
                                    new_chargers_list.append(index)
                                new_existing = charge_existing(vis_nodes, time_req, 'New')
                                if new_existing[1] == 'Full_trip':
                                    if new_existing[0] != -1:
                                        for temp in new_existing[0]:
                                            if temp[2] > 0:
                                                r['Departure'].iloc[temp[0]:] += temp[2]/60
                                                r['Arrival'].iloc[temp[0]:] += temp[2]/60
                                                vis_nodes['Schedule_changed'].iloc[temp[0]] = True
                                                Schedule_changed_indexes.append(temp[0])
                                            if temp[1] > 0:
                                                vis_nodes['charged_before'].iloc[temp[0]] = True
                                                
                                        Range_at_start = 0
                                        rem_dist = r['Distance'].sum()
                                        i = 0
                                        reverse_the_charge(j)
                                        for node_ind in new_chargers_list:
                                            if j == query:
                                                print('Charger removed from', temp_df.iloc[node_ind]['Node'])
                                            Chargers[temp_df.iloc[node_ind]['Node']] -= 1
                                            charger_status[temp_df.iloc[node_ind]['Node']].pop()
                                        vis_nodes.drop(list(vis_nodes.index), inplace = True)
                                        vis_nodes.reset_index(drop = True)
                                        vis_nodes.loc[0] = [j, r.iloc[0]['Depot'], 0, Range_at_start, r['Distance'].sum(), r.iloc[len(r) - 1]['Arrival'], r.iloc[0]['Departure'], \
                                                            min(math.ceil(((R - Range_at_start)/KPM_slow)/5)*5, math.ceil(((r.iloc[0]['Departure'] - r.iloc[len(r) - 1]['Arrival'])*60)/5)*5), False, False]
                                        Schedule_changed = True
                                        display(r)
                                        print("Hello world - 4")
                                        break
                                    new_charger_index = 'Done'
                                    break
                                elif new_existing[1] == 'Partial_trip':
                                    if j == query:
                                        print("Partial trip possible")
                                    for temp in new_existing[0]:
                                        vis_nodes['charged_before'].iloc[temp[0]] = True
                                        vis_nodes['Range_at_start'].iloc[temp[0]:] += (KPM*temp[1])
                                        vis_nodes["Range_at_start"] = np.where(vis_nodes["Range_at_start"] > R, R, vis_nodes["Range_at_start"])
                                    Range_at_start = min(Range_at_start + (KPM*new_existing[2]), R) - trip['Distance']
                                    new_charger_index = 'Not Done'
                                    break
                                elif new_existing[1] == 'Partial_with_changes':
                                    if j == query:
                                        print("Partial trip with Schedule changes possible")
                                    for temp in new_existing[0]:
                                        print(vis_nodes['Node'].iloc[temp[0]], vis_nodes['Start'].iloc[temp[0]], temp[2])
                                        if temp[2] > 0:
                                            r['Departure'].iloc[temp[0]:] += temp[2]/60
                                            r['Arrival'].iloc[temp[0]:] += temp[2]/60
                                            Schedule_changed_indexes.append(temp[0])
                                            print(Schedule_changed_indexes, "Dekhooo")
    #                                         vis_nodes['Schedule_changed'].iloc[temp[0]] = True
    #                                     if temp[1] > 0:
    #                                         vis_nodes['charged_before'].iloc[temp[0]] = True
    #                                     vis_nodes['Range_at_start'].iloc[temp[0]:] += (KPM*(temp[1] + temp[2]))
    #                                     vis_nodes["Range_at_start"] = np.where(vis_nodes["Range_at_start"] > R, R, vis_nodes["Range_at_start"])
    #                                 print(new_existing[2], '**Here')
                                    Range_at_start = 0
                                    rem_dist = r['Distance'].sum()
                                    i = 0
                                    display(Route_charging[Route_charging['Bus'] == query])
                                    reverse_the_charge(j)
    #                                 Schedule_changes.drop(list(Schedule_changes[Schedule_changes['Bus'] == j].index), inplace = True)
    #                                 Schedule_changes.reset_index(drop = True)
                                    for node_ind in new_chargers_list:
                                        if j == query:
                                            print('Charger removed from', temp_df.iloc[node_ind]['Node'])
                                        Chargers[temp_df.iloc[node_ind]['Node']] -= 1
                                        charger_status[temp_df.iloc[node_ind]['Node']].pop()
                                    vis_nodes.drop(list(vis_nodes.index), inplace = True)
                                    vis_nodes.reset_index(drop = True)
                                    vis_nodes.loc[0] = [j, r.iloc[0]['Depot'], 0, Range_at_start, r['Distance'].sum(), r.iloc[len(r) - 1]['Arrival'], r.iloc[0]['Departure'], \
                                                        min(math.ceil(((R - Range_at_start)/KPM_slow)/5)*5, math.ceil(((r.iloc[0]['Departure'] - r.iloc[len(r) - 1]['Arrival'])*60)/5)*5), False, False]
                                    Schedule_changed = True
                                    display(r)
                                    print("Hello world - 1")
                                    break
                                else:
                                    print("Threshold", threshold)
                                    print("new_chargers", new_chargers)
                                    if index == len(temp_df) - 1:
                                        for node_ind in new_chargers_list:
                                            if j == query:
                                                print('Charger removed from', temp_df.iloc[node_ind]['Node'])
                                            Chargers[temp_df.iloc[node_ind]['Node']] -= 1
                                            charger_status[temp_df.iloc[node_ind]['Node']].pop()
                                    elif time_avl > already_avl_time and new_chargers >= threshold:
                                        if j == query:
                                            print('Charger removed from', row['Node'])
                                        Chargers[row['Node']] -= 1
                                        charger_status[row['Node']].pop()
                                        new_chargers -= 1
                                        new_chargers_list.remove(index)
                    
                        if Schedule_changed == True:
                            print("Hello world - 2")
                            break
                                            
                        if new_charger_index != -1:
                            break

                    if(new_charger_index == 'Done'):
                        break
                    elif(new_charger_index == -1 and Schedule_changed == False):
                        if j == query:
                            print("Infeasible:", j)
                        Infeasible_Routes.append(j)
                        reverse_the_charge(j)
                        Schedule_changes.drop(list(Schedule_changes[Schedule_changes['Bus'] == j].index), inplace = True)
                        Schedule_changes.reset_index(drop = True)
                        break
                    elif new_charger_index != 'Not Done' and Schedule_changed == False:
                        charge_row = temp_df.iloc[new_charger_index]
                        if time_avl > already_avl_time:
                            charger_status[charge_row['Node']].append([True]*288)
                            Chargers[charge_row['Node']] += 1
                        start = math.ceil(charge_row['Start']*12)
                        end = math.ceil(charge_row['End']*12)
                        new_charge_time = 0
                        Range = charge_row['Range_at_start']
                        for ind in range(start, end):
                            if Range >= R or Range >= charge_row['rem_dist']:
                                break
                            Range = min(Range + (5*KPM), R)
                            if j == query:
                                print(ind)
                            charger_status[charge_row['Node']][-1][ind] = False
                            new_charge_time += 5

                        vis_nodes['Range_at_start'].iloc[vis_nodes_ind:] += (KPM*new_charge_time)
                        vis_nodes["Range_at_start"] = np.where(vis_nodes["Range_at_start"] > R, R, vis_nodes["Range_at_start"])
                        vis_nodes['charged_before'].iloc[vis_nodes_ind] = True
                        Route_charging.loc[len(Route_charging)] = [j, charge_row['Node'], len(charger_status[charge_row['Node']]), \
                        to_time_units(charge_row['Start']), new_charge_time, 0]
                        Range_at_start = min(Range_at_start + (KPM*new_charge_time), R) - trip['Distance']

                        if Range >= charge_row['rem_dist']:
                            break
                    
            else:
                Range_at_start -= trip['Distance']
            
            if Schedule_changed == True:
                print("Hello world - 3")
                continue
            
            rem_dist -= trip['Distance']
            
            if i < r.shape[0] - 1:
                vis_nodes.loc[len(vis_nodes)] = [j, end_node, Priority(end_node), Range_at_start, rem_dist, r.iloc[i]['Arrival'],\
                                             r.iloc[i+1]['Departure'], min(math.ceil(((R - Range_at_start)/KPM)/5)*5, math.ceil(((r.iloc[i+1]['Departure'] - r.iloc[i]['Arrival'])*60)/5)*5), False, False]
                if j == query:
                    print(r.iloc[i+1]['Departure'], r.iloc[i]['Arrival'])
                if (len(vis_nodes) - 1) in Schedule_changed_indexes:
                    print("Ha bhai kar diya", len(vis_nodes))
                    vis_nodes.at[len(vis_nodes)-1, 'Schedule_changed'] = True
            
            if j == query:
                display(vis_nodes)
            i += 1
        
        depot_charge_row = Route_charging[(Route_charging['Bus'] == j) & (Route_charging['Charged at'] == r.iloc[0]['Depot'])]
        
        depot_charge_row['Charge Start Time'] = depot_charge_row['Charge Start Time'].apply(to_decimal)
        temp_start = depot_charge_row[depot_charge_row['Charge Start Time'] >= vis_nodes['Start'].iloc[0]]['Charge Start Time'].min()
        if j == query:
            print(j, temp_start, r.iloc[len(r) - 1]['Arrival'], vis_nodes['Start'].iloc[0])
        if temp_start < r.iloc[len(r) - 1]['Arrival'] and \
            temp_start >= vis_nodes['Start'].iloc[0]:
            print("***", j, r.iloc[0]['Depot'])
            for index in range(len(depot_charge_row)):
                initial = math.ceil(depot_charge_row.iloc[index]['Charge Start Time']*12)
                final = math.ceil((depot_charge_row.iloc[index]['Charge Start Time'] \
                           + (depot_charge_row.iloc[index]['Charging duration(Minutes)']/60))*12)
                for tstamp in range(initial, final):
                    if tstamp >= 288:
                        tstamp = tstamp - 288
                    charger_status[r.iloc[0]['Depot']][depot_charge_row.iloc[index]['Charger_gun'] - 1][tstamp] = True

            Route_charging.drop(list(depot_charge_row.index), inplace = True)
            Route_charging.reset_index(drop = True, inplace = True)
            vis_nodes.loc[0] = [j, r.iloc[0]['Depot'], 0, 0, r['Distance'].sum(), r.iloc[len(r) - 1]['Arrival'], r.iloc[0]['Departure'], \
                            min(math.ceil((R/KPM_slow)/5)*5, math.ceil(((r.iloc[0]['Departure'] - r.iloc[len(r) - 1]['Arrival'])*60)/5)*5), False, False]
            Charge(vis_nodes.iloc[0], 0, 1e9)
            
num_charger_df = pd.DataFrame(columns = ["Location", "Number of Chargers"])
for i in Chargers:
    if(i not in depots and Chargers[i] != 0):
        num_charger_df.loc[len(num_charger_df)] = [i, Chargers[i]]
        
Route_charging.to_excel("Depot wise bus charging schedule.xlsx",index=False)
    

C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:105: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  m=pd.to_datetime(datetime.datetime.utcfromtimestamp(l).strftime('%Y-%m-%d %H:%M:%S')).time()
C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:115: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  s=pd.to_datetime(df_1['New_start_time'].astype("str"))
C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:148: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp

DOWN


C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:784: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_31["dummy_id"]=f1
C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:1156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_31.rename(columns={"dummy_id":"Route","trip_number":"Trip number","Start_location":"From","End_location":"To"},inplace=True)
C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:1174: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys wil

0
Saurashtra University 0
Saurashtra University 0
Saurashtra University 0
Saurashtra University 0


C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:1748: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depot_charge_row['Charge Start Time'] = depot_charge_row['Charge Start Time'].apply(to_decimal)
C:\Users\Welcome\AppData\Local\Temp\ipykernel_10148\1479224302.py:1748: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  depot_charge_row['Charge Start Time'] = depot_charge_row['Charge Start Time'].apply(to_decimal)
